In [1]:
] activate "/Users/anarres/Documents/projects/BioXP"

 Activating environment at `~/Documents/projects/BioXP/Project.toml`


In [2]:
using BioXP
using ProgressMeter
# using BenchmarkTools

### Inputs

In [8]:
## Organism JSONs from ecg (JGI)
# input_dir = "data/input/rids/"
input_dir = "data/input/rids-methanogens/"

## Master file from ecg (with dgs added from `add_dgs_to_master` .py files)
rstructs_path = "data/input/rstructs/master_from_redges-og-submission.json"

## User defined seeds/targets
seeds_path = "data/input/seeds/encel_papers_2019.json"
targets_path = "data/input/seeds/seeds.json"
sid_name = "Contains_KEGGID_P"
tid_name = "targets_Freilich09";

## Path to write to
# write_dir = joinpath("data","output",sid_name)
write_dir = joinpath("data","output",sid_name*"-methanogens")

if !ispath(write_dir)
    mkpath(write_dir)
end

"data/output/Contains_KEGGID_P-methanogens"

### Run expansion

Double check number of accessible threads

In [9]:
Threads.nthreads()

10

Looping such that seeds are logically assigned

In [10]:
sids = readkeyedids(seeds_path)[sid_name]
tids = readkeyedids(targets_path)[tid_name]
rstructs = readmaster(rstructs_path)

for path in readdir(input_dir)
    
    org_dir = joinpath(input_dir,path)
    
    if isdir(org_dir) &  !startswith(path,".") 
        
        if !ispath(joinpath(write_dir,path))
            mkpath(joinpath(write_dir,path))
        end
        
        p = Progress(length(readdir(org_dir)),desc="$path")
        
        println(org_dir)
        Threads.@threads for (i,fname) in collect(enumerate(readdir(org_dir)))
#             Random.seed!(seedoffset+i)
            if !startswith(fname,".") 
                rids = readids(joinpath(org_dir,fname))
                write_path = joinpath(write_dir,basename(org_dir),fname)
                x, t, cids, X, Y = expand(rstructs,rids,sids,tids,write_path) ## new results
                next!(p)
            end
        end
    end
end

data/input/rids-methanogens/archaea


archaea 50%|██████████████████████                      |  ETA: 0:00:00

### Format output

In [11]:
simple_output_dir = write_dir #joinpath("data","output",sid_name)
    
for path in readdir(simple_output_dir)
    org_dir = joinpath(simple_output_dir,path)
    if !startswith(path,".") 
        formatbioxpoutput(org_dir)
    end
end